# 🌳 Tutorial: Deep Learning for Dependency Parsing

![Dependency Tree](https://upload.wikimedia.org/wikipedia/commons/8/8e/Thistreeisillustratingdependencyrelations.jpg)

## Welcome to the Fascinating World of Dependency Parsing! 🔗

In this comprehensive tutorial, you'll learn:
- 🌳 What is dependency parsing and why it matters in NLP
- 🧠 How transformer models like BERT can capture syntactic relationships
- 🤖 Building neural networks for syntactic analysis
- 💻 Hands-on implementation with PyTorch and HerBERT
- 📊 Evaluation metrics and tree construction algorithms
- 🧪 Interactive exercises to build your skills

By the end, you'll be ready to implement sophisticated dependency parsing using neural networks!


## 📚 Table of Contents

1. [🎓 Understanding Dependency Parsing](#1--understanding-dependency-parsing)
2. [🧮 Mathematical Foundation](#2--mathematical-foundation)
3. [🔧 Setting Up the Environment](#3--setting-up-the-environment)
4. [🏗️ Classical vs Neural Approaches](#4--classical-vs-neural-approaches)
5. [🤖 BERT and Contextual Embeddings](#5--bert-and-contextual-embeddings)
6. [🧠 Building the Neural Models](#6--building-the-neural-models)
7. [🎯 Training Strategy](#7--training-strategy)
8. [🌳 Tree Construction Algorithm](#8--tree-construction-algorithm)
9. [📊 Evaluation Metrics](#9--evaluation-metrics)
10. [🎮 Interactive Exercises](#10--interactive-exercises)
11. [🚀 Advanced Techniques](#11--advanced-techniques)
12. [📖 Summary and Next Steps](#12--summary-and-next-steps)


## 1. 🎓 Understanding Dependency Parsing

### What is Dependency Parsing?

Imagine you're reading a sentence and trying to understand **who does what to whom** 🤔. Dependency parsing is exactly that - it's the process of analyzing the grammatical structure of sentences to understand the relationships between words.

**Example**: "Maria bought a beautiful red car yesterday."

In this sentence:
- **"bought"** is the main verb (root)
- **"Maria"** is the subject of "bought"
- **"car"** is the object of "bought"
- **"beautiful"** and **"red"** modify "car"
- **"yesterday"** modifies "bought"

### Dependency Tree Structure

```
         bought
    ______|______
   |             |
 Maria          car
             ____|____
            |    |    |
           a  beautiful red
                      |
                  yesterday
```

### Key Concepts:

- **Head**: The governing word in a relationship (e.g., "bought" governs "Maria")
- **Dependent**: The word that depends on another (e.g., "Maria" depends on "bought")
- **Root**: The main word of the sentence (usually the main verb)
- **Tree Structure**: Each word has exactly one head (except root), forming a tree

### Why is This Important?

🤖 **Machine Translation**: Understanding structure helps translate correctly  
🔍 **Question Answering**: Finding relationships between entities  
📝 **Text Summarization**: Identifying key information structures  
🧠 **Sentiment Analysis**: Understanding what modifies what  
📊 **Information Extraction**: Finding structured data in text


## 2. 🧮 Mathematical Foundation

### The Dependency Parsing Problem

Given a sentence $S = w_1, w_2, ..., w_n$ with $n$ words, we want to find:

1. **Root word** $r \in \{1, 2, ..., n\}$
2. **Dependency edges** $E = \{(h_i, d_i)\}$ where $h_i$ is the head and $d_i$ is the dependent

### Tree Constraints

A valid dependency tree must satisfy:

1. **Exactly one root**: $|\{w_i : head(w_i) = \text{NULL}\}| = 1$
2. **Connected**: Every word is reachable from root
3. **Acyclic**: No cycles in the dependency graph
4. **Single head**: Each non-root word has exactly one head

### Distance Matrix

For tree construction, we define the **distance matrix** $D$ where:

$$D_{i,j} = \text{shortest path length between words } w_i \text{ and } w_j$$

Key properties:
- $D_{i,i} = 0$ (distance to self)
- $D_{i,j} = D_{j,i}$ (symmetric)
- $D_{i,j} = 1$ if there's a direct edge between $w_i$ and $w_j$

### Depth from Root

The **depth** of word $w_i$ is:

$$depth(w_i) = D_{root, i}$$

- Root has depth 0
- Direct children of root have depth 1
- And so on...

### The Neural Approach

Instead of using traditional parsing algorithms, we'll use **neural networks** to:

1. **Predict edge probabilities**: $P(edge_{i,j}) = f_{edge}(emb_i, emb_j)$
2. **Predict root probabilities**: $P(root_i) = f_{root}(emb_i)$
3. **Construct tree**: Use greedy algorithm with predicted probabilities


## 3. 🔧 Setting Up the Environment

Let's start by importing all the necessary libraries and setting up our environment for dependency parsing.


In [ ]:
# Essential imports for dependency parsing
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
import os
from typing import List, Tuple, Dict, Set

# PyTorch for neural networks
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# Transformers for BERT/HerBERT
from transformers import AutoModel, AutoTokenizer
from tqdm import tqdm

# Suppress warnings for cleaner output
import warnings

warnings.filterwarnings("ignore")

# Set up device - CPU is fine for this tutorial
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Using device: {device}")

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

print("✅ Environment setup complete!")
print(f"📦 PyTorch version: {torch.__version__}")
print(f"🖥️  CUDA available: {torch.cuda.is_available()}")

In [ ]:
# Sample sentence structures for demonstration
class Sentence:
    """Represents a sentence with words"""

    def __init__(self, words: List[str]):
        self.words = words

    def __len__(self):
        return len(self.words)

    def __str__(self):
        return " ".join(self.words)


class ParsedSentence(Sentence):
    """Represents a sentence with dependency structure"""

    def __init__(self, words: List[str], edges: List[Tuple[int, int]], root: int):
        super().__init__(words)
        self.edges = edges  # List of (head, dependent) pairs
        self.root = root  # Index of root word

    def get_sorted_edges(self):
        """Return edges sorted by head index"""
        return sorted(self.edges)

    def pretty_print(self):
        """Print a simple visualization of the dependency tree"""
        print(f"Root: {self.words[self.root]} (index {self.root})")
        print("Edges:")
        for head, dep in self.get_sorted_edges():
            print(f"  {self.words[head]} -> {self.words[dep]}")


# Create sample sentences for demonstration
sample_sentences = [
    ParsedSentence(
        words=["Maria", "bought", "a", "car", "."],
        edges=[(1, 0), (1, 3), (3, 2), (1, 4)],  # bought -> Maria, car, .
        root=1,  # "bought" is root
    ),
    ParsedSentence(
        words=["The", "cat", "sleeps", "peacefully", "."],
        edges=[(2, 1), (1, 0), (2, 3), (2, 4)],  # sleeps -> cat, peacefully, .
        root=2,  # "sleeps" is root
    ),
]

print("📚 Sample sentences created:")
for i, sent in enumerate(sample_sentences):
    print(f"\n{i+1}. {sent}")
    sent.pretty_print()

## 4. 🏗️ Classical vs Neural Approaches

Before diving into neural methods, let's understand how dependency parsing was traditionally done.

### Classical Approaches

#### 1. Transition-Based Parsing
- **Idea**: Process words left-to-right, making parsing decisions
- **Actions**: SHIFT (move to next word), REDUCE (create dependency)
- **Problem**: Greedy decisions can lead to errors

#### 2. Graph-Based Parsing
- **Idea**: Score all possible edges, find best tree
- **Algorithm**: Maximum Spanning Tree (MST)
- **Problem**: Scoring function is hand-crafted

#### 3. Rule-Based Parsing
- **Idea**: Use grammatical rules to determine dependencies
- **Example**: "Adjectives modify nouns"
- **Problem**: Rules don't capture all linguistic phenomena

### Why Neural Approaches? 🤖

❌ **Traditional Problems**:
- Hand-crafted features
- Limited context understanding
- Difficulty handling ambiguity
- Poor generalization

✅ **Neural Advantages**:
- **Automatic feature learning**: No manual feature engineering
- **Rich representations**: Contextual embeddings from BERT
- **Flexible architectures**: Can model complex relationships
- **End-to-end training**: Optimize for final task


In [ ]:
# Let's implement a simple distance calculation function
def calculate_tree_distances(parsed_sentence: ParsedSentence) -> np.ndarray:
    """
    Calculate the distance matrix for a dependency tree.

    This function computes the shortest path distance between every pair of words
    in the dependency tree. This is a fundamental operation in dependency parsing.

    Args:
        parsed_sentence: A sentence with known dependency structure

    Returns:
        numpy array of shape (n, n) where n is the number of words
        distances[i][j] = shortest path from word i to word j
    """
    n = len(parsed_sentence)
    distances = np.zeros((n, n))

    # Build adjacency list from edges
    neighbors = [[] for _ in range(n)]
    for head, dep in parsed_sentence.get_sorted_edges():
        neighbors[head].append(dep)
        neighbors[dep].append(head)  # Undirected graph for distance calculation

    # DFS to compute distances from each node
    def dfs(start, current, visited, dist):
        visited[current] = True
        for neighbor in neighbors[current]:
            if not visited[neighbor]:
                distances[start][neighbor] = dist + 1
                dfs(start, neighbor, visited, dist + 1)

    # Compute distances from each starting node
    for i in range(n):
        visited = [False] * n
        distances[i][i] = 0  # Distance to self is 0
        dfs(i, i, visited, 0)

    return distances


# Test our distance calculation
print("🧮 Testing distance calculation:")
for i, sent in enumerate(sample_sentences):
    distances = calculate_tree_distances(sent)
    print(f"\nSentence {i+1}: {sent}")
    print("Distance matrix:")
    print(distances.astype(int))

    # Show depths (distances from root)
    root_distances = distances[sent.root]
    print(f"Depths from root '{sent.words[sent.root]}': {root_distances.astype(int)}")

## 5. 🤖 BERT and Contextual Embeddings

The key to our neural approach is using **HerBERT** (Polish BERT) to get rich, contextual representations of words.

### Why HerBERT?

🧠 **Contextual Understanding**: Unlike traditional word embeddings, BERT considers the entire sentence context  
🇵🇱 **Polish Language**: HerBERT is specifically trained on Polish text  
🎯 **Syntactic Knowledge**: BERT's hidden layers capture syntactic relationships  
🔄 **Subword Tokenization**: Handles unknown words gracefully

### The BERT Architecture

```
Input: "Maria kupuje samochód"
   ↓ Tokenization
[CLS] Maria kupuje samochód [SEP]
   ↓ Embedding Layer
768-dimensional vectors
   ↓ 12 Transformer Layers
Contextual representations
   ↓ Layer 7 (our choice)
Rich syntactic features
```

### Why Layer 7?

Research shows that **middle layers** of BERT capture the best syntactic information:
- **Early layers** (1-3): Surface features, POS tags
- **Middle layers** (4-8): Syntactic relationships, dependencies  
- **Late layers** (9-12): Semantic features, task-specific info

Layer 7 is our sweet spot for dependency parsing! 🎯

### 🔧 Handling Subword Tokenization

**The Challenge**: BERT tokenizes words into subwords (WordPiece)
- "programowanie" → ["program", "##owanie"]
- "najdłuższy" → ["naj", "##dłuż", "##szy"]

**The Solution**: We need to **aggregate** subword embeddings back to word-level

**Aggregation Strategies**:
1. **Average**: Take mean of all subword embeddings ✅
2. **First**: Use only the first subword embedding
3. **Last**: Use only the last subword embedding
4. **Max**: Take element-wise maximum

We'll use **averaging** as it preserves information from all subwords.


In [ ]:
# Load HerBERT model and tokenizer
print("📦 Loading HerBERT model...")
tokenizer = AutoTokenizer.from_pretrained("allegro/herbert-base-cased")
bert_model = AutoModel.from_pretrained("allegro/herbert-base-cased")
bert_model.eval()  # Set to evaluation mode
print("✅ HerBERT loaded successfully!")


def get_bert_embeddings(sentences: List[str], layer_idx: int = 7):
    """
    Extract embeddings from HerBERT for a list of sentences.

    This function demonstrates the complete BERT processing pipeline:
    1. Tokenization: Split sentences into subwords
    2. Encoding: Convert to input IDs with special tokens
    3. Forward pass: Get hidden states from all layers
    4. Extraction: Get embeddings from specified layer

    Args:
        sentences: List of sentences as strings
        layer_idx: Which BERT layer to extract (7 is good for syntax)

    Returns:
        List of tensors, each containing embeddings for one sentence
    """
    # Tokenize and encode all sentences
    encoded = tokenizer.batch_encode_plus(
        sentences, return_tensors="pt", padding=True, truncation=True, max_length=128
    )

    # Get embeddings from BERT
    with torch.no_grad():
        outputs = bert_model(**encoded, output_hidden_states=True)

    # Extract embeddings from specified layer
    layer_embeddings = outputs.hidden_states[
        layer_idx + 1
    ]  # +1 because layer 0 is embeddings

    # Process each sentence separately
    sentence_embeddings = []
    for i in range(len(sentences)):
        # Get attention mask for this sentence
        attention_mask = encoded["attention_mask"][i].bool()

        # Extract real tokens (excluding padding)
        real_embeddings = layer_embeddings[i, attention_mask]

        # Remove [CLS] and [SEP] tokens
        clean_embeddings = real_embeddings[1:-1]  # Remove first and last tokens

        sentence_embeddings.append(clean_embeddings)

    return sentence_embeddings


# Test BERT embeddings
test_sentences = ["Maria kupuje samochód .", "Kot śpi spokojnie ."]
print(f"🧪 Testing BERT embeddings on: {test_sentences}")

embeddings = get_bert_embeddings(test_sentences)
for i, (sent, emb) in enumerate(zip(test_sentences, embeddings)):
    words = sent.split()
    print(f"\nSentence {i+1}: {sent}")
    print(f"  Words: {len(words)}")
    print(f"  Embeddings shape: {emb.shape}")
    print(f"  Embedding dimension: {emb.shape[-1]}")

    # Note: We might need to handle subword tokenization
    # For now, let's see what we get
    if emb.shape[0] != len(words):
        print(f"  ⚠️  Mismatch: {emb.shape[0]} embeddings vs {len(words)} words")
        print(f"  This is normal - BERT uses subword tokenization!")

In [ ]:
def tokenize_and_align(sentence: str, tokenizer):
    """
    Tokenize a sentence and create alignment between words and subwords.

    This function helps us understand how BERT's tokenization relates to
    the original words in our sentence.

    Args:
        sentence: Original sentence string
        tokenizer: HerBERT tokenizer

    Returns:
        dict with original words, subword tokens, and alignment mapping
    """
    # Split into words
    words = sentence.split()

    # Tokenize each word separately to understand the mapping
    word_to_subwords = []
    all_subwords = []

    for word in words:
        # Tokenize individual word (add space prefix for proper tokenization)
        subwords = tokenizer.tokenize(" " + word)[0:]  # Keep space token for Polish
        if not subwords:  # Handle edge case
            subwords = tokenizer.tokenize(word)

        word_to_subwords.append(subwords)
        all_subwords.extend(subwords)

    # Create alignment: subword_idx -> word_idx
    subword_to_word = []
    for word_idx, subwords in enumerate(word_to_subwords):
        subword_to_word.extend([word_idx] * len(subwords))

    return {
        "words": words,
        "subwords": all_subwords,
        "word_to_subwords": word_to_subwords,
        "subword_to_word": subword_to_word,
    }


def aggregate_subword_embeddings(
    embeddings: torch.Tensor,
    subword_to_word: List[int],
    num_words: int,
    method: str = "mean",
) -> torch.Tensor:
    """
    Aggregate subword embeddings to word-level embeddings.

    Args:
        embeddings: Tensor of subword embeddings [num_subwords, embed_dim]
        subword_to_word: List mapping subword indices to word indices
        num_words: Number of original words
        method: Aggregation method ('mean', 'first', 'last', 'max')

    Returns:
        Tensor of word embeddings [num_words, embed_dim]
    """
    embed_dim = embeddings.shape[-1]
    word_embeddings = torch.zeros(num_words, embed_dim)

    for word_idx in range(num_words):
        # Find all subwords belonging to this word
        subword_indices = [i for i, w in enumerate(subword_to_word) if w == word_idx]

        if subword_indices:
            subword_embs = embeddings[subword_indices]

            if method == "mean":
                word_embeddings[word_idx] = torch.mean(subword_embs, dim=0)
            elif method == "first":
                word_embeddings[word_idx] = subword_embs[0]
            elif method == "last":
                word_embeddings[word_idx] = subword_embs[-1]
            elif method == "max":
                word_embeddings[word_idx] = torch.max(subword_embs, dim=0)[0]

    return word_embeddings


# Test tokenization and alignment
test_sentence = "Programowanie jest fascynujące ."
print(f"🔍 Analyzing tokenization for: '{test_sentence}'")

alignment = tokenize_and_align(test_sentence, tokenizer)
print(f"\n📝 Original words: {alignment['words']}")
print(f"🔤 Subwords: {alignment['subwords']}")
print(f"📊 Word-to-subwords mapping:")
for i, (word, subwords) in enumerate(
    zip(alignment["words"], alignment["word_to_subwords"])
):
    print(f"  {i}: '{word}' → {subwords}")

print(f"\n🔗 Subword-to-word alignment: {alignment['subword_to_word']}")

# Test aggregation
embeddings = get_bert_embeddings([test_sentence])[0]
print(f"\n📊 Subword embeddings shape: {embeddings.shape}")

word_embeddings = aggregate_subword_embeddings(
    embeddings, alignment["subword_to_word"], len(alignment["words"])
)
print(f"📊 Word embeddings shape: {word_embeddings.shape}")
print(
    f"✅ Successfully aggregated {embeddings.shape[0]} subwords to {word_embeddings.shape[0]} words!"
)

## 6. 🧠 Building the Neural Models

Now comes the exciting part! We'll build two neural networks to solve dependency parsing:

1. **Distance Model**: Predicts if two words should be connected by an edge
2. **Depth Model**: Predicts which word should be the root of the tree

### Our Neural Architecture Strategy

#### Distance Model 🔗
- **Input**: Concatenated embeddings of two words (768 × 2 = 1536 dimensions)
- **Task**: Binary classification - edge or no edge?
- **Output**: Probability that words i and j are connected

#### Depth Model 🌳  
- **Input**: Single word embedding (768 dimensions)
- **Task**: Binary classification - root or not root?
- **Output**: Probability that word i is the root

### Why This Approach?

✅ **Simplicity**: Two focused models are easier to train than one complex model  
✅ **Interpretability**: We can analyze what each model learns  
✅ **Flexibility**: Can train with different strategies and hyperparameters  
✅ **Efficiency**: Smaller models train faster

### Architecture Details

Both models follow a similar structure:
- **Input Layer**: Takes BERT embeddings
- **Hidden Layer**: 256 neurons with LeakyReLU activation
- **Dropout**: 30% for regularization
- **Output**: Single neuron with sigmoid activation


In [ ]:
class DistanceModel(nn.Module):
    """
    Neural network that predicts whether two words should be connected by an edge.

    Architecture:
    - Input: Concatenated embeddings of two words (768*2 = 1536 dims)
    - Hidden: 256 neurons with LeakyReLU and dropout
    - Output: Single probability (sigmoid)

    This model learns to identify syntactic relationships between word pairs!
    """

    def __init__(self, input_dim=768 * 2, hidden_dim=256, dropout_rate=0.3):
        super(DistanceModel, self).__init__()

        self.network = nn.Sequential(
            # Input layer: concatenated word embeddings -> hidden layer
            nn.Linear(input_dim, hidden_dim),
            # LeakyReLU: allows small gradients for negative values
            nn.LeakyReLU(negative_slope=0.01),
            # Regularization: randomly drop 30% of neurons during training
            nn.Dropout(p=dropout_rate),
            # Output layer: hidden -> probability
            nn.Linear(hidden_dim, 1),
            # Sigmoid: maps any real number to [0,1] probability
            nn.Sigmoid(),
        )

    def forward(self, x):
        """
        Forward pass through the network.

        Args:
            x: Tensor of shape [batch_size, 1536] - concatenated word pairs

        Returns:
            Tensor of shape [batch_size, 1] - edge probabilities
        """
        return self.network(x)


class DepthModel(nn.Module):
    """
    Neural network that predicts whether a word should be the root of the tree.

    Architecture:
    - Input: Single word embedding (768 dims)
    - Hidden: 256 neurons with LeakyReLU and dropout
    - Output: Single probability (sigmoid)

    This model learns to identify which words are most likely to be sentence roots!
    """

    def __init__(self, input_dim=768, hidden_dim=256, dropout_rate=0.3):
        super(DepthModel, self).__init__()

        self.network = nn.Sequential(
            # Input layer: single word embedding -> hidden layer
            nn.Linear(input_dim, hidden_dim),
            # LeakyReLU activation
            nn.LeakyReLU(negative_slope=0.01),
            # Regularization dropout
            nn.Dropout(p=dropout_rate),
            # Output layer: hidden -> probability
            nn.Linear(hidden_dim, 1),
            # Sigmoid activation
            nn.Sigmoid(),
        )

    def forward(self, x):
        """
        Forward pass through the network.

        Args:
            x: Tensor of shape [batch_size, 768] - word embeddings

        Returns:
            Tensor of shape [batch_size, 1] - root probabilities
        """
        return self.network(x)


# Let's create and test our models
print("🏗️ Creating neural network models...")

# Initialize models
distance_model = DistanceModel()
depth_model = DepthModel()

print(f"✅ Distance Model created!")
print(f"   Parameters: {sum(p.numel() for p in distance_model.parameters()):,}")
print(f"   Input: {768*2} dims (two word embeddings)")
print(f"   Output: 1 dim (edge probability)")

print(f"\n✅ Depth Model created!")
print(f"   Parameters: {sum(p.numel() for p in depth_model.parameters()):,}")
print(f"   Input: {768} dims (single word embedding)")
print(f"   Output: 1 dim (root probability)")

# Test with dummy data
print(f"\n🧪 Testing models with dummy data...")

# Test distance model
dummy_pair = torch.randn(4, 768 * 2)  # Batch of 4 word pairs
distance_predictions = distance_model(dummy_pair)
print(f"Distance model output shape: {distance_predictions.shape}")
print(
    f"Distance predictions (should be [0,1]): {distance_predictions.squeeze().detach().numpy()}"
)

# Test depth model
dummy_words = torch.randn(5, 768)  # Batch of 5 words
depth_predictions = depth_model(dummy_words)
print(f"Depth model output shape: {depth_predictions.shape}")
print(
    f"Depth predictions (should be [0,1]): {depth_predictions.squeeze().detach().numpy()}"
)

print(f"\n🎉 Both models working correctly!")

## 7. 🎯 Training Strategy

Now that we have our models, let's understand how to train them effectively for dependency parsing.

### The Data Challenge 📊

**Imbalanced Data Problem**:
- Most word pairs are **NOT** connected (negative examples)
- Only a few word pairs **ARE** connected (positive examples)
- Ratio can be 1:20 or worse!

**Root Detection Problem**:
- Only **ONE** word per sentence is the root
- All other words are **NOT** roots
- Even more imbalanced than edges!

### Our Training Solutions 💡

#### 1. Balanced Sampling for Distance Model
```python
# Instead of using all negative examples:
positive_pairs = pairs_with_edges      # Few examples
negative_pairs = pairs_without_edges   # Many examples

# Sample equal numbers:
balanced_negatives = random.sample(negative_pairs, len(positive_pairs))
training_data = positive_pairs + balanced_negatives
```

#### 2. Balanced Sampling for Depth Model
```python
# Instead of using all non-root words:
root_words = [word for word in sentence if word.is_root]     # 1 example
non_root_words = [word for word in sentence if not word.is_root]  # Many examples

# Sample equal numbers:
balanced_non_roots = random.sample(non_root_words, len(root_words))
training_data = root_words + balanced_non_roots
```

### Why This Works 🎯

✅ **Prevents bias**: Model doesn't just predict "no edge" all the time  
✅ **Faster training**: Smaller, balanced datasets train quicker  
✅ **Better learning**: Model sees equal examples of both classes  
✅ **Improved metrics**: Better precision and recall on positive class

### Training Loop Overview

For each epoch:
1. **Shuffle** and **balance** the data
2. **Forward pass** through model
3. **Compute loss** (Binary Cross Entropy)
4. **Backward pass** (gradient computation)
5. **Update weights** (optimizer step)
6. **Evaluate** on validation set


## 8. 🌳 Tree Construction Algorithm

Once our models are trained, we need to convert their predictions into a valid dependency tree. This is where the magic happens!

### The Challenge 🤔

Our models give us:
- **Edge probabilities**: P(edge between word i and word j)
- **Root probabilities**: P(word i is root)

But we need:
- **Valid tree**: Exactly n-1 edges, connected, acyclic
- **Single root**: Exactly one root word
- **Connected**: Every word reachable from root

### Our Greedy Tree Construction Algorithm 🛠️

```python
def construct_dependency_tree(edge_probabilities, root_probabilities):
    # Step 1: Choose root (highest root probability)
    root = argmax(root_probabilities)
    
    # Step 2: Initialize tree with root
    nodes_in_tree = {root}
    edges = []
    
    # Step 3: Greedily add edges
    for _ in range(n - 1):  # Need exactly n-1 edges
        best_edge = None
        best_prob = -infinity
        
        # Find best edge from tree to outside
        for node_in_tree in nodes_in_tree:
            for candidate in nodes_not_in_tree:
                prob = edge_probabilities[node_in_tree][candidate]
                if prob > best_prob:
                    best_prob = prob
                    best_edge = (node_in_tree, candidate)
        
        # Add best edge to tree
        edges.append(best_edge)
        nodes_in_tree.add(best_edge[1])
    
    return edges, root
```

### Why This Algorithm Works ✅

1. **Guarantees valid tree**: Always produces exactly n-1 edges
2. **Maintains connectivity**: Always extends from existing tree
3. **Prevents cycles**: Never connects within the tree
4. **Uses model predictions**: Chooses highest probability edges and root
5. **Efficient**: O(n²) time complexity

### Visual Example 👁️

```
Sentence: "The cat sleeps peacefully"
Root probs: [0.1, 0.2, 0.9, 0.1]  → Choose "sleeps" as root

Step 1: Tree = {sleeps}
Best edge: sleeps → cat (prob=0.8)
Tree = {sleeps, cat}

Step 2: Tree = {sleeps, cat}  
Best edge: cat → The (prob=0.7)
Tree = {sleeps, cat, The}

Step 3: Tree = {sleeps, cat, The}
Best edge: sleeps → peacefully (prob=0.6)
Tree = {sleeps, cat, The, peacefully}

Final tree: sleeps → cat → The, sleeps → peacefully
```


In [ ]:
def parse_sentence_demo(sentence_words, distance_model, depth_model):
    """
    Demonstration of the complete parsing pipeline.

    This function shows how all the pieces fit together:
    1. Get word embeddings from BERT
    2. Use models to predict edge and root probabilities
    3. Construct tree with greedy algorithm
    """
    print(f"🔍 Parsing sentence: {' '.join(sentence_words)}")

    # Step 1: Get word embeddings (simplified - assumes perfect word alignment)
    sentence_str = " ".join(sentence_words)
    embeddings = get_bert_embeddings([sentence_str])[0]

    # For demo, let's assume we have word-level embeddings
    # In practice, you'd need the subword aggregation from earlier
    n_words = len(sentence_words)

    # Create dummy embeddings if mismatch (for demo purposes)
    if embeddings.shape[0] != n_words:
        print(
            f"   📝 Note: Using dummy embeddings for demo ({embeddings.shape[0]} subwords → {n_words} words)"
        )
        embeddings = torch.randn(n_words, 768)

    # Step 2: Get predictions from models
    print("   🤖 Getting model predictions...")

    with torch.no_grad():
        # Root probabilities for each word
        root_probs = depth_model(embeddings).squeeze()

        # Edge probabilities for all word pairs
        edge_probs = torch.zeros(n_words, n_words)
        for i in range(n_words):
            for j in range(n_words):
                if i != j:  # Don't predict self-edges
                    # Concatenate embeddings of word pair
                    pair_embedding = torch.cat([embeddings[i], embeddings[j]])
                    edge_probs[i][j] = distance_model(
                        pair_embedding.unsqueeze(0)
                    ).item()

    # Step 3: Construct tree greedily
    print("   🌳 Constructing dependency tree...")

    # Choose root (word with highest root probability)
    root_idx = torch.argmax(root_probs).item()
    print(
        f"   👑 Root selected: '{sentence_words[root_idx]}' (prob: {root_probs[root_idx]:.3f})"
    )

    # Greedy tree construction
    nodes_in_tree = {root_idx}
    edges = []

    for step in range(n_words - 1):
        best_edge = None
        best_prob = -1

        # Find best edge from tree to outside
        for node_in_tree in nodes_in_tree:
            for candidate in range(n_words):
                if candidate not in nodes_in_tree:
                    prob = edge_probs[node_in_tree][candidate].item()
                    if prob > best_prob:
                        best_prob = prob
                        best_edge = (node_in_tree, candidate)

        if best_edge:
            edges.append(best_edge)
            nodes_in_tree.add(best_edge[1])
            head, dep = best_edge
            print(
                f"   🔗 Step {step+1}: {sentence_words[head]} → {sentence_words[dep]} (prob: {best_prob:.3f})"
            )

    # Create result
    result = ParsedSentence(sentence_words, edges, root_idx)

    print(f"\n   ✅ Final tree:")
    result.pretty_print()

    return result


# Test our parsing pipeline
test_words = ["The", "cat", "sleeps", "peacefully"]
print("🚀 Testing complete parsing pipeline:")
result = parse_sentence_demo(test_words, distance_model, depth_model)

print(f"\n📊 Tree statistics:")
print(f"   - Root: {result.words[result.root]}")
print(f"   - Edges: {len(result.edges)}")
print(f"   - Expected edges: {len(result.words) - 1}")
print(f"   - Tree valid: {len(result.edges) == len(result.words) - 1}")

## 9. 📊 Evaluation Metrics

Understanding how to measure parsing quality is crucial for developing good models.

### Primary Metrics 🎯

#### 1. UUAS (Unlabeled Undirected Attachment Score)
- **What it measures**: Percentage of correctly identified edges
- **Formula**: `UUAS = (correct_edges / total_edges) × 100`
- **Range**: 0% to 100% (higher is better)

```python
def calculate_uuas(true_edges, predicted_edges):
    true_set = set(true_edges)
    pred_set = set(predicted_edges)
    
    # Convert directed edges to undirected for comparison
    true_undirected = set()
    for head, dep in true_set:
        true_undirected.add((min(head, dep), max(head, dep)))
    
    pred_undirected = set()
    for head, dep in pred_set:
        pred_undirected.add((min(head, dep), max(head, dep)))
    
    correct = len(true_undirected & pred_undirected)
    total = len(true_undirected)
    
    return correct / total if total > 0 else 0.0
```

#### 2. Root Placement Accuracy
- **What it measures**: Percentage of correctly identified sentence roots
- **Formula**: `Root_Acc = (correct_roots / total_sentences) × 100`
- **Range**: 0% to 100% (higher is better)

```python
def calculate_root_accuracy(true_roots, predicted_roots):
    correct = sum(1 for true, pred in zip(true_roots, predicted_roots) if true == pred)
    total = len(true_roots)
    return correct / total if total > 0 else 0.0
```

### Final Score Calculation 🏆

The problem uses a weighted combination:

```python
def calculate_final_score(root_placement, uuas):
    def scale(x, lower=0.5, upper=0.85):
        scaled = min(max(x, lower), upper)
        return (scaled - lower) / (upper - lower)
    
    return scale(root_placement) + scale(uuas)
```

**Interpretation**:
- Score range: 0.0 to 2.0
- Each metric contributes 0-1 points
- Performance below 50% gets 0 points
- Performance above 85% gets full points
- Linear scaling between 50% and 85%

### Why These Metrics? 🤔

✅ **UUAS**: Measures structural understanding  
✅ **Root Placement**: Critical for tree validity  
✅ **Scaling**: Encourages meaningful performance  
✅ **Balance**: Both metrics matter equally


In [ ]:
# Let's implement the evaluation metrics
def calculate_uuas(
    true_sentence: ParsedSentence, pred_sentence: ParsedSentence
) -> float:
    """Calculate UUAS score for a single sentence"""
    true_edges = set(true_sentence.get_sorted_edges())
    pred_edges = set(pred_sentence.get_sorted_edges())

    # Convert to undirected edges
    true_undirected = set()
    for head, dep in true_edges:
        true_undirected.add((min(head, dep), max(head, dep)))

    pred_undirected = set()
    for head, dep in pred_edges:
        pred_undirected.add((min(head, dep), max(head, dep)))

    if len(true_undirected) == 0:
        return 1.0 if len(pred_undirected) == 0 else 0.0

    correct = len(true_undirected & pred_undirected)
    return correct / len(true_undirected)


def calculate_final_score(root_placement: float, uuas: float) -> float:
    """Calculate final score as defined in the problem"""

    def scale(x, lower=0.5, upper=0.85):
        scaled = min(max(x, lower), upper)
        return (scaled - lower) / (upper - lower)

    return scale(root_placement) + scale(uuas)


# Test evaluation on our sample sentences
print("📊 Testing evaluation metrics:")

# Create a "predicted" version of our first sample sentence
true_sentence = sample_sentences[0]  # "Maria bought a car ."
# Let's create a slightly wrong prediction for testing
pred_sentence = ParsedSentence(
    words=["Maria", "bought", "a", "car", "."],
    edges=[
        (1, 0),
        (1, 3),
        (1, 2),
        (1, 4),
    ],  # Different from true: (1,2) instead of (3,2)
    root=1,  # Same root
)

print(f"\nTrue sentence: {true_sentence}")
true_sentence.pretty_print()

print(f"\nPredicted sentence: {pred_sentence}")
pred_sentence.pretty_print()

# Calculate metrics
uuas_score = calculate_uuas(true_sentence, pred_sentence)
root_correct = true_sentence.root == pred_sentence.root
final_score = calculate_final_score(1.0 if root_correct else 0.0, uuas_score)

print(f"\n📈 Evaluation Results:")
print(f"   UUAS Score: {uuas_score:.3f} ({uuas_score*100:.1f}%)")
print(f"   Root Correct: {root_correct} ({100 if root_correct else 0}%)")
print(f"   Final Score: {final_score:.3f}/2.0")

# Show which edges were correct/incorrect
true_edges = set(true_sentence.get_sorted_edges())
pred_edges = set(pred_sentence.get_sorted_edges())

print(f"\n🔍 Edge Analysis:")
print(f"   True edges: {true_edges}")
print(f"   Pred edges: {pred_edges}")
print(f"   Correct: {true_edges & pred_edges}")
print(f"   Missing: {true_edges - pred_edges}")
print(f"   Extra: {pred_edges - true_edges}")

## 10. 🎮 Interactive Exercises

Now it's your turn to experiment and learn! Try these challenges to deepen your understanding.

### 🎯 Exercise 1: Model Architecture Exploration

Try modifying the neural network architectures and observe the effects:

1. **Change layer sizes**: Try 128, 512, or 1024 hidden neurons
2. **Add more layers**: Create deeper networks  
3. **Try different activations**: ReLU, ELU, or Swish
4. **Experiment with dropout**: 0.1, 0.5, or 0.7

**Questions to consider**:
- How does model size affect training speed?
- Do deeper networks perform better?
- What happens with very high dropout rates?

### 🎯 Exercise 2: Aggregation Strategy Comparison

Compare different methods for aggregating subword embeddings:

1. **Mean**: Average all subword embeddings ✅ (current)
2. **First**: Use only the first subword
3. **Last**: Use only the last subword  
4. **Max**: Element-wise maximum
5. **Attention**: Learned weighted combination

**Implementation hint**:
```python
# Try implementing attention-based aggregation
class AttentionAggregation(nn.Module):
    def __init__(self, embed_dim=768):
        super().__init__()
        self.attention = nn.Linear(embed_dim, 1)
    
    def forward(self, subword_embeddings):
        # subword_embeddings: [num_subwords, embed_dim]
        weights = torch.softmax(self.attention(subword_embeddings), dim=0)
        return torch.sum(weights * subword_embeddings, dim=0)
```

### 🎯 Exercise 3: Tree Construction Algorithms

Experiment with different tree construction strategies:

1. **Greedy** (current): Always add highest probability edge
2. **Beam search**: Keep top-k partial trees
3. **Maximum Spanning Tree**: Use edge weights
4. **Random sampling**: Sample edges based on probabilities

### 🎯 Exercise 4: Training Strategies

Try different approaches to handle class imbalance:

1. **Different sampling ratios**: 1:1, 1:2, 1:5 positive:negative
2. **Weighted loss**: Give positive examples higher weight
3. **Focal loss**: Focus learning on hard examples
4. **Curriculum learning**: Start with easy examples

### 🎯 Exercise 5: Error Analysis

Analyze what your models get wrong:

1. **Edge type analysis**: Which syntactic relationships are hardest?
2. **Sentence length**: Performance vs sentence length
3. **Root word patterns**: What makes good/bad root predictions?
4. **Common errors**: Most frequent mistake patterns


In [ ]:
# 🧪 Experiment Playground - Try different model architectures!


def create_modified_distance_model(
    hidden_dim=256, num_layers=1, activation="leaky_relu", dropout_rate=0.3
):
    """
    Create a distance model with customizable architecture.

    Args:
        hidden_dim: Size of hidden layers
        num_layers: Number of hidden layers (1-3)
        activation: 'relu', 'leaky_relu', 'elu', or 'swish'
        dropout_rate: Dropout probability
    """
    layers = []

    # Input layer
    layers.append(nn.Linear(768 * 2, hidden_dim))

    # Activation function
    if activation == "relu":
        layers.append(nn.ReLU())
    elif activation == "leaky_relu":
        layers.append(nn.LeakyReLU(negative_slope=0.01))
    elif activation == "elu":
        layers.append(nn.ELU())
    elif activation == "swish":
        layers.append(nn.SiLU())  # SiLU is the same as Swish

    layers.append(nn.Dropout(dropout_rate))

    # Additional hidden layers
    for _ in range(num_layers - 1):
        layers.append(nn.Linear(hidden_dim, hidden_dim))
        if activation == "relu":
            layers.append(nn.ReLU())
        elif activation == "leaky_relu":
            layers.append(nn.LeakyReLU(negative_slope=0.01))
        elif activation == "elu":
            layers.append(nn.ELU())
        elif activation == "swish":
            layers.append(nn.SiLU())
        layers.append(nn.Dropout(dropout_rate))

    # Output layer
    layers.append(nn.Linear(hidden_dim, 1))
    layers.append(nn.Sigmoid())

    return nn.Sequential(*layers)


# Test different architectures
architectures_to_test = [
    {"name": "Small", "hidden_dim": 128, "num_layers": 1},
    {"name": "Standard", "hidden_dim": 256, "num_layers": 1},  # Current
    {"name": "Large", "hidden_dim": 512, "num_layers": 1},
    {"name": "Deep", "hidden_dim": 256, "num_layers": 3},
]

print("🔬 Testing different model architectures:")
for arch in architectures_to_test:
    model = create_modified_distance_model(
        **{k: v for k, v in arch.items() if k != "name"}
    )

    # Count parameters
    num_params = sum(p.numel() for p in model.parameters())

    # Test forward pass
    dummy_input = torch.randn(10, 768 * 2)
    with torch.no_grad():
        output = model(dummy_input)

    print(
        f"   {arch['name']:>8}: {num_params:>6,} params, output shape: {output.shape}"
    )

print(f"\n💡 Experiment ideas:")
print(f"   - Which architecture would train fastest?")
print(f"   - Which might generalize best?")
print(f"   - How does parameter count relate to performance?")

## 11. 🚀 Advanced Techniques

Ready to take your dependency parsing to the next level? Here are some advanced techniques used in state-of-the-art systems.

### 1. Multi-Layer BERT Features 🎭

Instead of using just layer 7, combine features from multiple layers:

```python
def get_multi_layer_embeddings(sentences, layers=[4, 7, 10]):
    """Extract and combine features from multiple BERT layers"""
    all_embeddings = []
    
    for layer in layers:
        layer_embs = get_bert_embeddings(sentences, layer_idx=layer)
        all_embeddings.append(layer_embs)
    
    # Concatenate or average across layers
    combined = torch.cat(all_embeddings, dim=-1)  # Concatenation
    # OR: combined = torch.mean(torch.stack(all_embeddings), dim=0)  # Average
    
    return combined
```

### 2. Biaffine Attention 🎯

Advanced edge scoring using biaffine transformations:

```python
class BiaffineAttention(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.head_mlp = nn.Linear(768, hidden_dim)
        self.dep_mlp = nn.Linear(768, hidden_dim)
        self.biaffine = nn.Linear(hidden_dim, hidden_dim)
        
    def forward(self, embeddings):
        head_repr = self.head_mlp(embeddings)  # [seq_len, hidden_dim]
        dep_repr = self.dep_mlp(embeddings)    # [seq_len, hidden_dim]
        
        # Biaffine scoring: head^T * W * dep for all pairs
        scores = torch.einsum('ih,hj,jk->ik', 
                             head_repr, 
                             self.biaffine.weight, 
                             dep_repr.transpose(0, 1))
        return scores
```

### 3. Constrained Training 🔒

Ensure model predictions form valid trees during training:

```python
def tree_constraint_loss(edge_probs, true_edges):
    """Add penalty for violating tree constraints"""
    # Penalty for disconnected components
    connectivity_loss = compute_connectivity_penalty(edge_probs)
    
    # Penalty for cycles
    cycle_loss = compute_cycle_penalty(edge_probs)
    
    # Standard BCE loss
    bce_loss = F.binary_cross_entropy(edge_probs, true_edges)
    
    return bce_loss + 0.1 * connectivity_loss + 0.1 * cycle_loss
```

### 4. Ensemble Methods 🎪

Combine multiple models for better performance:

```python
class EnsembleParser:
    def __init__(self, models):
        self.models = models
    
    def predict(self, sentence):
        # Get predictions from all models
        all_edge_probs = []
        all_root_probs = []
        
        for model in self.models:
            edge_p, root_p = model.predict(sentence)
            all_edge_probs.append(edge_p)
            all_root_probs.append(root_p)
        
        # Average predictions
        avg_edge_probs = torch.mean(torch.stack(all_edge_probs), dim=0)
        avg_root_probs = torch.mean(torch.stack(all_root_probs), dim=0)
        
        return avg_edge_probs, avg_root_probs
```

### 5. Data Augmentation 📈

Increase training data diversity:

```python
def augment_sentence(sentence):
    """Create variations of training sentences"""
    augmentations = []
    
    # Word dropout: randomly remove words
    if len(sentence) > 3:
        indices = random.sample(range(1, len(sentence)-1), k=1)
        new_words = [w for i, w in enumerate(sentence.words) if i not in indices]
        augmentations.append(create_sentence_from_words(new_words))
    
    # Synonym replacement (if you have a synonym dictionary)
    # Word reordering (for some languages)
    
    return augmentations
```

### 📚 Research Directions

- **Graph Neural Networks**: Use GNNs for iterative refinement
- **Transformer Parsers**: End-to-end parsing with transformers
- **Cross-lingual Transfer**: Train on multiple languages
- **Few-shot Learning**: Adapt to new domains with little data


## 12. 📖 Summary and Next Steps

Congratulations! 🎉 You've learned how to use deep learning for dependency parsing!

### What You've Learned:

1. **🌳 Dependency Parsing Fundamentals**:
   - Understanding syntactic tree structures
   - Head-dependent relationships and tree constraints
   - Distance matrices and depth calculations

2. **🤖 Neural Approach**:
   - Using HerBERT for contextual word embeddings
   - Two-model architecture: distance and depth prediction
   - Handling subword tokenization and aggregation

3. **💻 Implementation Skills**:
   - PyTorch model architecture design
   - Balanced training for imbalanced data
   - Greedy tree construction algorithms

4. **📊 Evaluation & Analysis**:
   - UUAS and root placement metrics
   - Error analysis and performance debugging
   - Comparative evaluation strategies

### For the Solution Implementation:

You now have all the knowledge to implement the complete solution! The key components are:

```python
def your_dependency_parser(sentence, tokenizer, bert_model, distance_model, depth_model):
    # 1. Extract word embeddings from HerBERT
    embeddings = get_word_embeddings([sentence], tokenizer, bert_model)[0]
    
    # 2. Predict edge probabilities
    edge_probs = predict_all_edges(embeddings, distance_model)
    
    # 3. Predict root probabilities  
    root_probs = depth_model(embeddings)
    
    # 4. Construct tree greedily
    tree = construct_dependency_tree(edge_probs, root_probs)
    
    return tree
```

### 🚀 Key Success Factors:

- **Quality embeddings**: Proper subword aggregation is crucial
- **Balanced training**: Handle class imbalance carefully
- **Valid trees**: Ensure your algorithm always produces valid trees
- **Hyperparameter tuning**: Learning rates, epochs, and model sizes matter
- **Evaluation**: Monitor both UUAS and root placement

### 🎯 Expected Performance Targets:

Based on the scoring function, aim for:
- **UUAS**: 70-85% (unlabeled attachment score)
- **Root Placement**: 75-90% (correct root identification)
- **Final Score**: 1.0-2.0 points

### 📚 Useful Resources:

- 🛠️ [HerBERT Documentation](https://huggingface.co/allegro/herbert-base-cased)
- 📖 [Universal Dependencies](https://universaldependencies.org/) - Annotation guidelines
- 📑 [Dependency Parsing Survey](https://www.aclweb.org/anthology/J08-4003.pdf)
- 🎓 [Stanford CS224N](http://web.stanford.edu/class/cs224n/) - NLP Course

### 🔥 Advanced Topics to Explore:

- **Graph-based parsing**: Maximum spanning tree algorithms
- **Transition-based parsing**: Arc-standard and arc-eager systems
- **Neural architectures**: BiLSTMs, attention mechanisms, transformers
- **Cross-lingual parsing**: Universal Dependencies and multilingual models

**Good luck with your implementation!** 🌟

Remember: The key insight is that neural networks can learn complex syntactic patterns from contextual embeddings, making them much more powerful than traditional feature-based approaches. Your models will learn to identify both local syntactic patterns and long-range dependencies automatically!
